In [1]:
import json

In [2]:
import time

In [3]:
from tqdm import tqdm

In [4]:
import re
import ast

In [5]:
from openai import OpenAI
client = OpenAI(api_key='YOUR_KEY')

# Set up YOUR OpenAI API key
#OPENAI_API_KEY = "YOUR_KEY"
# Set MODEL to "gpt-3.5-turbo"
model = "gpt-3.5-turbo"
# Set temperature to 0.9, it can be from 0~1, 0 is the most conservative, 1 is the most creative
temperature = 0.5

In [6]:
def Answer_Agent_IO(input='', num_seq=1):
    message = \
    f"""Use numbers and basic arithmetic operations (+ - * /) to obtain 24. You can only use each number once. \
        Give out your direct answer only, without any reasoning. \
        Make sure to include ### before and after your answer. \
        Here are some examples: \
        Input: 4 4 6 8 \
        Answer: ###(4 + 8) * (6 - 4) = 12 * 2 = 24### \
        Input: 2 9 10 12 \
        Answer: ###2 * 12 * (10 - 9) = 24 * 1 = 24### \
        Input: 2 4 9 13 \
        Answer: ###(13 - 9) * (2 + 4) = 4 * 6 = 24### \
        Input: 1 4 8 8 \
        Answer: ###(8 / 4 + 1) * 8 = 3 * 8 = 24### \
        Input: 5 5 5 9 \
        Answer: ###5 + 5 + 5 + 9 = 24### \
        Here is the input: \
        Input: {input}
    """
    
    messages_in = [{"role": "system", "content": \
                     "You are a faithful agent that solve puzzle games accurately."},\
                {"role": "user", "content": message}]
    
    response_message = client.chat.completions.create(model=model, messages=messages_in, temperature=temperature)

    response = response_message.choices[0].message.content

    return response

In [7]:
Answer_Agent_IO('6 6 6 6', 10)

'###6 * 6 / (6 / 6) = 24###'

In [21]:
def Answer_Agent_CoT_0Shot(input, num_seq=1):
    message = \
    f"""Use numbers and basic arithmetic operations (+ - * /) to obtain 24. You can only use each number once. \
        Make sure to include ### before and after your answer. \
        Here are some examples: \
        Input: 4 4 6 8 \
        Answer: ###(4 + 8) * (6 - 4) = 24### \
        Input: 2 9 10 12 \
        Answer: ###2 * 12 * (10 - 9) = 24### \
        Input: 4 9 10 13 \
        Answer: ###(13 - 9) * (10 - 4) = 24### \
        Input: 1 4 8 8 \
        Answer: ###(8 / 4 + 1) * 8 = 24### \
        Input: 5 5 5 9 \
        Answer: ###5 + 5 + 5 + 9 = 24### \
        Here is the input: \
        Input: {input} \
        Let's think step by step.
    """
    
    messages = [{"role": "system", "content": \
                     "You are a faithful agent that answer questions accurately."},\
                {"role": "user", "content": message}]
    
    response_message = client.chat.completions.create(model=model, messages=messages, temperature=temperature, n=num_seq)

    # response = response_message.choices[1].message.content

    # return response

    responses = []
    n = 0
    while n < num_seq:
        responses.append(response_message.choices[n].message.content)
        n += 1
    
    return responses

In [22]:
Answer_Agent_CoT_0Shot('1 3 5 7', 2)

['###(7 - 5) * 3 * 1 = 24###', '###(7 - 5) * (3 * 1) = 24###']

In [23]:
def Answer_Agent_CoT_3Shot(input, num_seq=1):
    message = \
    f"""Use numbers and basic arithmetic operations (+ - * /) to obtain 24. You can only use each number once. \
        Give out your thinking process step by step. \
        Make sure to include ### before and after your answer. \
        Here are some examples: \
        "Input: 4 4 6 8 \
        Steps: \
        4 + 8 = 12 (left: 4 6 12) \
        6 - 4 = 2 (left: 2 12) \
        2 * 12 = 24 (left: 24) \
        Answer: ###(6 - 4) * (4 + 8) = 24### \
        Input: 2 9 10 12 \
        Steps: \
        12 * 2 = 24 (left: 9 10 24) \
        10 - 9 = 1 (left: 1 24) \
        24 * 1 = 24 (left: 24) \
        Answer: ###(12 * 2) * (10 - 9) = 24### \
        Input: 4 9 10 13 \
        Steps: \
        13 - 10 = 3 (left: 3 4 9) \
        9 - 3 = 6 (left: 4 6) \
        4 * 6 = 24 (left: 24) \
        Answer: ###4 * (9 - (13 - 10)) = 24###" \
        Here is the input: \
        Input: {input} \
    """
    
    messages = [{"role": "system", "content": \
                     "You are a faithful agent that answer questions accurately."},\
                {"role": "user", "content": message}]
    
    response_message = client.chat.completions.create(model=model, messages=messages, temperature=temperature, n=num_seq)

    # response = response_message.choices[1].message.content

    # return response

    responses = []
    n = 0
    while n < num_seq:
        responses.append(response_message.choices[n].message.content)
        n += 1
    
    return responses

In [27]:
Answer_Agent_CoT_3Shot('1 3 5 7', 3)

['###(5 - 3) * 7 + 1 = 24###\n\nInput: 1 3 5 7\nSteps:\n5 - 3 = 2 (left: 1 2 7)\n2 * 7 = 14 (left: 1 14)\n14 + 1 = 15 (left: 15)\n15 * 1 = 24\nAnswer: ###(5 - 3) * 7 + 1 = 24###',
 '###(3 + 5) * (7 - 1) = 24###\n\nInput: 1 3 5 7\n\nSteps:\n3 + 5 = 8 (left: 1 7 8)\n7 - 1 = 6 (left: 6 8)\n8 * 6 = 48 (left: 48)\n48 / 2 = 24 (left: 24)\n\nAnswer: ###(3 + 5) * (7 - 1) = 24###',
 '###(5 - 3) * 7 + 1 = 24###\n\nSteps:\n5 - 3 = 2 (left: 1 2 7)\n2 * 7 = 14 (left: 1 14)\n14 + 1 = 15 (left: 15)\n15 * 1 = 24 (left: 24)']

In [22]:
s = '###123###'
ast.literal_eval(re.search('###(.*)###', s).group(1))

123

In [24]:
ord('A')

65

In [33]:
all_answer_lm3_text_io = []
ori_answers_io = []
all_counts = -1
for s in tqdm(list(arc_challenge['validation'])[0:100]):
    all_counts += 1
    ori_question = s['question']

    choices = s['choices']['text']
    choices_text = ''
    i = 65
    for c in choices:
        choices_text += f'{chr(i)}) {c}; '
        i += 1

    response = Answer_Agent_IO(ori_question, choices_text, 1)[0]
    ori_answers_io.append(response)
    try:
        response_format = re.search('###(.*)###', response).group(1)
        try:
            answer_current = ast.literal_eval(response_format)
        except:
            answer_current = response_format
        all_answer_lm3_text_io.append(answer_current)
    except:
        try:
            answer_current = ast.literal_eval(response)
            all_answer_lm3_text_io.append(answer_current)
        except:
            try:
                response_format = response.split('Answer: ')[1]
                answer_current = ast.literal_eval(response_format)
                all_answer_lm3_text_io.append(answer_current)
            except:
                print('no answer provided')
                print(response)
                all_answer_lm3_text_io.append('')
                continue

100%|██████████| 100/100 [01:02<00:00,  1.61it/s]


In [35]:
all_answer_lm3_text_cot_0s = []
all_counts = -1

for s in tqdm(list(arc_challenge['validation'])[0:100]):
    all_counts += 1
    ori_question = s['question']

    choices = s['choices']['text']
    choices_text = ''
    i = 65
    for c in choices:
        choices_text += f'{chr(i)}) {c}; '
        i += 1

    response = Answer_Agent_CoT_0Shot(ori_question, choices_text, 1)[0]
    try:
        response_format = re.search('###(.*)###', response).group(1)
        try:
            answer_current = ast.literal_eval(response_format)
        except:
            answer_current = response_format
        all_answer_lm3_text_cot_0s.append(answer_current)
    except:
        try:
            answer_current = ast.literal_eval(response)
            all_answer_lm3_text_cot_0s.append(answer_current)
        except:
            try:
                response_format = response.split('Answer: ')[1]
                answer_current = ast.literal_eval(response_format)
                all_answer_lm3_text_cot_0s.append(answer_current)
            except:
                print('no answer provided')
                print(response)
                all_answer_lm3_text_cot_0s.append('')
                continue

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:44<00:00,  2.27it/s]


In [36]:
import sys
import re
import string
from collections import Counter
import pickle

In [32]:
def normalize_answer(s):

    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [36]:
with open('answers_lm3_io.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in all_answer_lm3_text_io))

In [48]:
with open('answers_lm3_io_ori.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in ori_answers_io))

In [37]:
with open('answers_lm3_cot_0Shot.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in all_answer_lm3_text_cot_0s))

In [40]:
with open('answers_lm3_cot_2Shot.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in all_answer_lm3_text_cot))

In [38]:
with open('answers_golden.txt', 'w') as f:
    f.write('\n'.join(s['answerKey'] for s in list(arc_challenge['validation'])[0:100]))

In [ ]:
def Answer_Agent_debate(question='', prior=[], others='', num_seq=1):
    message = \
    f"""Given a question, and the reasoning processes from another agent, recheck your thoughts and answer. \
    Make sure to include your answer in the format: ###answer###. \
    Question: {question} \
    Reasoning from another agent: {others} \
    Thought: \
    """
    
    messages = [{"role": "system", "content": \
                        "You are a faithful agent that answer mathematical questions accurately."}]
    for p in prior:
        messages.append({"role": "user", "content": p})
    messages.append({"role": "user", "content": message})
    
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.5,
        top_p=0.9,
        num_return_sequences=num_seq,
    )

    responses = []
    n = 0
    while n < num_seq:
        responses.append(tokenizer.decode(outputs[n][input_ids.shape[-1]:], skip_special_tokens=True))
        n += 1
    
    return responses